<center><font color='steelblue'> <font size = "5,5">CORPORACIÓN UNIVERSITARIA MINUTO DE DIOS </font></center><br>
<center><font color='steelblue'> <font size = "5">FACULTAD DE INGENIERÍA</font></center><br>
<center><font color='steelblue'> <font size = "4">PROGRAMA INGENIERÍA DE SISTEMAS</font></center><br>
<center><font color='steelblue'> <font size = "3">CURSO BASES DE DATOS MASIVAS</font></center><br>
<center><font color="yellow" size = "4" face = "small fonts">Proyecto Modular - Mojo</font></center>

<center><font color="olive" size = "4" face = "small fonts">DATOS DE LOS PARTICIPANTES DEL GRUPO</font></center>

<font color="yellow" size = "4" face = "small fonts">NRC: 80132</font></center><br>
<font color="yellow" size = "4" face = "small fonts">Nombres: Laura Tatiana Bernal Yanquen, Daniel Yesid Casallas Páez</font></center><br>
<font color="yellow" size = "4" face = "small fonts">ID:857437</font></center><br>

<font color="yellow" size = "4" face = "small fonts">El taller retoma algunas de las instrucciones utilizadas a través del curso de Bases de Datos Masivas, tener en cuenta seguir los pasos requeridos.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">Al Cargar los archivos dispuestos en la carpeta Poryecto final BDM se debe utilizar sentencias que no fijen el path, este debe se dinámico en caso de que se creen nuevos paquetes</font></center><br>

<font color="red" size = "4" face = "small fonts">El ejercicio consta de seguir el cuaderno y  realizar las tareas solicitadas, Utilizar y citar la documentación propia de cada una de las herramientas utilizadas para realizar el tratamiento a la data.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">1. Configuración e importe de las librerias.</font></center><br>

In [47]:
# Esta extencion permite que jupyter detecte cambios externos y recargue sin tener que reiniciar VSC
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
# Importar la librearia os para poder interactuar el sistema donde se esta ejecutando los archivos
import os
import requests
import re
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import FuncFormatter
import json

<font color="yellow" size = "4" face = "small fonts">2. Configure y levante los contenedores necesarios para la actividad, recuerde que la arquitectura respeta a 3 contenedores con una distribucion de DBMongo en cada una de ellas, un contenedor que tiene configurado Mojolicious con todas sus dependencias junto con una base de datos Relacional la cual sera escogida a su gusto.</font></center><br>

In [ ]:
# Moverse a la carpeta data donde estan los archivos para lanzar el docker
os.chdir("../data")
# Crear y ejecutar el docker-compose.yml
os.system("docker-compose up --build")

<font color="yellow" size = "4" face = "small fonts">3. Una vez tenga el contenedor arriba cargue los datos .Json, uno por cada contenedor DBMongo.</font></center><br>

<font color="yellow" size = "4" face = "small fonts">Tenga en cuenta que debe tener la replica de los formatos `JSon` en cada contenedor</font></center><br>

In [49]:
# Con estos comandos se deben cargar los datos .json dentro de las mongoDB sin necesidad de una funcion.

# Comando para importar personas.json
os.system("docker cp ../data/personas.json core_container:/app/data/personas.json")

# Comando para importar articulos.json
os.system("docker cp ../data/articulos.json core_container:/app/data/articulos.json")

# Comando para importar ventas.json
os.system("docker cp ../data/ventas.json core_container:/app/data/ventas.json") 

0

<font color="yellow" size = "4" face = "small fonts">Se debe colocar el puerto expuesto del contenedor `Mojo`, para llamar la función y cargar los datos</font></center><br>

In [50]:
# Cargar data a las Mongo a con la funcion
requests.get("http://localhost:8080/load_data")

<Response [200]>

<font color="yellow" size = "4" face = "small fonts">4. Muestre los datos que fueron almacenados en cada una de las distribuciones de BDMongo.</font></center><br>

In [ ]:
# Verificar conteo de documentos en la colección personas
os.system("")

# Verificar conteo de documentos en la colección articulos
os.system("")

# Verificar conteo de documentos en la colección ventas
os.system("")

<font color="yellow" size = "4" face = "small fonts">Consultar a tráves del puerto expuesto la data que se encuentran en las colecciones personas, articulos y ventas de DBMongo</font></center><br>

In [ ]:
# Ver personas en MongoDB
personas = requests.get("http://localhost:8080/mongo/personas")
print(json.dumps(personas.json(), ensure_ascii=False, indent=2))

In [ ]:
# Ver articulos en MongoDB
articulos = requests.get("http://localhost:8080/mongo/articulos")
print(json.dumps(articulos.json(), ensure_ascii=False, indent=2))

In [ ]:
# Ver ventas en MongoDB
ventas = requests.get("http://localhost:8080/mongo/ventas")
print(json.dumps(ventas.json(), ensure_ascii=False, indent=2))

<font color="yellow" size = "4" face = "small fonts">5. Verifique la estructura y tipo de los datos almacenados en las distribuciones DBMongo, genere una ETL para almacenar cada una de las distribuciones en la base de datos Relacional que configuro con anterioridad, recuerde que cada distribucion debe ir en una tabla relacional respetando la integridad referencial.</font></center><br>

In [ ]:
# Funciones en Python para obtener datos de MongoDB para articulos
def limpiar_valor_numerico(valor):
    
    if valor is None:
        return None
    
    # Convertir a string y limpiar espacios
    valor_str = str(valor).strip()
    
    # Eliminar caracteres no numéricos (excepto el signo negativo al inicio)
    valor_limpio = re.sub(r'[^\d-]', '', valor_str)
    
    try:
        return int(valor_limpio)
    except (ValueError, TypeError):
        return None
    


# Función para verificar y limpiar idArticulo y cantidadArticulo donde debe ser un dato entero, tener en cuenta elimiar el id que proporciona 
# DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.
def filtrar_y_limpiar_articulos(datos):
    # Definir los campos que necesitan ser numéricos
    campos_a_limpiar = ['idArticulo', 'cantidadArticulo']
    datos_limpios = []
    
    # Primera fase: Limpiar datos
    for documento in datos:
        # Eliminar el _id de MongoDB
        if '_id' in documento:
            del documento['_id']
        
        campos_validos = True
        
        # Validar y limpiar campos numéricos
        for campo in campos_a_limpiar:
            valor_limpio = limpiar_valor_numerico(documento.get(campo))
            
            if valor_limpio is None:
                campos_validos = False
                break
            
            documento[campo] = valor_limpio
        
        # Limpiar precioArticulo (puede ser float)
        if 'precioArticulo' in documento:
            try:
                precio_str = str(documento['precioArticulo']).strip()
                documento['precioArticulo'] = float(re.sub(r'[^\d.-]', '', precio_str))
            except (ValueError, TypeError):
                campos_validos = False
        
        if campos_validos:
            datos_limpios.append(documento)
    
    # Segunda fase: Consolidar artículos duplicados
    articulos_dict = {}
    
    for articulo in datos_limpios:
        id_articulo = articulo['idArticulo']
        
        if id_articulo not in articulos_dict:
            # Primer registro de este artículo
            articulos_dict[id_articulo] = {
                'idArticulo': id_articulo,
                'nombreArticulo': articulo.get('nombreArticulo', ''),
                'cantidadArticulo': articulo['cantidadArticulo'],
                'precioArticulo': articulo.get('precioArticulo', 0),
                'suma_precios': articulo.get('precioArticulo', 0),
                'contador': 1
            }
        else:
            # Artículo duplicado: sumar cantidad y precios
            articulos_dict[id_articulo]['cantidadArticulo'] += articulo['cantidadArticulo']
            articulos_dict[id_articulo]['suma_precios'] += articulo.get('precioArticulo', 0)
            articulos_dict[id_articulo]['contador'] += 1
    
    # Tercera fase: Calcular promedio de precios
    articulos_consolidados = []
    for articulo in articulos_dict.values():
        articulo['precioArticulo'] = round(articulo['suma_precios'] / articulo['contador'], 2)
        # Eliminar campos auxiliares
        del articulo['suma_precios']
        del articulo['contador']
        articulos_consolidados.append(articulo)
    
    return articulos_consolidados


# Uso de la función
response = requests.get("http://localhost:8080/mongo/articulos")
articulos = response.json()
articulos_limpios = filtrar_y_limpiar_articulos(articulos)
print(json.dumps(articulos_limpios, ensure_ascii=False, indent=2))

# Insertar en SQLite
for articulo in articulos_limpios:
    try:
        response = requests.post("http://localhost:8080/sqlite/articulos", json=articulo)
        if response.status_code != 200:
            print(f"Error al insertar artículo {articulo.get('idArticulo')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar artículo: {e}")

In [ ]:
# Obtener datos de MongoDB para personas, escriba un ETL donde haga la validación de numeroDocumento y telefono, asegurando que solo contengan valores 
# numéricos, tener en cuenta elimiar el id que proporciona DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.

def filtrar_limpiar_personas(personas, ventas):
    """
    ETL para limpiar y validar datos de personas y ventas.
    Primero valida los números de documento sin limpiar asociados a ventas.
    Agrega identificadores temporales para rastrear ventas originales.
    Maneja duplicados de numeroDocumento y asegura integridad referencial.
    """
    # Paso 1: Identificar números de documento asociados a ventas (sin limpiar)
    compradores_en_ventas_raw = set()
    
    for venta in ventas:
        id_comprador_raw = venta.get('idComprador')
        if id_comprador_raw is not None:
            compradores_en_ventas_raw.add(str(id_comprador_raw))
    
    print(f"Números de documento sin limpiar en ventas: {len(compradores_en_ventas_raw)}")
    
    # Paso 2: Limpiar datos de ventas y agregar identificador temporal
    ventas_limpias = []
    compradores_en_ventas = set()
    
    for idx, venta in enumerate(ventas):
        # Eliminar _id de MongoDB
        if '_id' in venta:
            del venta['_id']
        
        # Agregar identificador temporal para rastreo
        venta['_temp_id_venta'] = f"VENTA_{idx}"
        
        # Validar campos numéricos
        id_venta = limpiar_valor_numerico(venta.get('idVenta'))
        id_comprador = limpiar_valor_numerico(venta.get('idComprador'))
        id_articulo = limpiar_valor_numerico(venta.get('idArticulo'))
        cantidad = limpiar_valor_numerico(venta.get('cantidadProductos'))
        
        if None in [id_venta, id_comprador, id_articulo, cantidad]:
            continue
        
        # Limpiar precioTotal
        try:
            precio_total = float(re.sub(r'[^\d.-]', '', str(venta.get('precioTotal', 0))))
        except (ValueError, TypeError):
            continue
        
        venta['idVenta'] = id_venta
        venta['idComprador'] = id_comprador
        venta['_temp_doc_original'] = id_comprador  # Guardar documento original
        venta['idArticulo'] = id_articulo
        venta['cantidadProductos'] = cantidad
        venta['precioTotal'] = round(precio_total, 2)
        
        ventas_limpias.append(venta)
        compradores_en_ventas.add(id_comprador)
    
    print(f"Números de documento limpios en ventas: {len(compradores_en_ventas)}")
    
    # Paso 3: Limpiar y preparar datos de personas (solo las que tienen ventas)
    personas_limpias = []
    
    for idx, persona in enumerate(personas):
        # Eliminar _id de MongoDB
        if '_id' in persona:
            del persona['_id']
        
        # Verificar si el documento sin limpiar está en ventas
        num_doc_raw = str(persona.get('numeroDocumento', ''))
        if num_doc_raw not in compradores_en_ventas_raw:
            continue
        
        # Validar y limpiar numeroDocumento (debe ser numérico)
        num_doc = limpiar_valor_numerico(persona.get('numeroDocumento'))
        if num_doc is None or num_doc not in compradores_en_ventas:
            continue
        
        # Agregar identificador temporal para rastreo
        persona['_temp_id_persona'] = f"PERSONA_{idx}_DOC_{num_doc}"
        
        # Validar y limpiar telefono (debe ser numérico)
        telefono = persona.get('telefono', '')
        if telefono:
            telefono_limpio = re.sub(r'[^\d]', '', str(telefono).strip())
            persona['telefono'] = telefono_limpio if telefono_limpio else None
        else:
            persona['telefono'] = None
        
        persona['numeroDocumento'] = num_doc
        persona['_temp_doc_original'] = num_doc  # Guardar documento original
        personas_limpias.append(persona)
    
    # Paso 4: Resolver duplicados en personas
    personas_dict = {}
    nuevo_documento = 100000000  # Número de documento para reasignar
    mapeo_documentos = {}  # Mapea: (doc_original, temp_id_persona) -> nuevo_doc
    
    for persona in personas_limpias:
        num_doc = persona['numeroDocumento']
        temp_id = persona['_temp_id_persona']
        
        if num_doc not in personas_dict:
            # Primera vez que vemos este documento
            personas_dict[num_doc] = persona
            mapeo_documentos[(num_doc, temp_id)] = num_doc
        else:
            # Duplicado encontrado: reasignar número de documento
            while nuevo_documento in personas_dict or nuevo_documento in compradores_en_ventas:
                nuevo_documento += 1
            
            print(f"Duplicado detectado: documento {num_doc} -> reasignado a {nuevo_documento}")
            
            # Actualizar ventas que referencian este duplicado específico
            for venta in ventas_limpias:
                if venta['_temp_doc_original'] == num_doc:
                    # Aquí deberías tener lógica adicional para determinar
                    # a cuál persona pertenece esta venta específica
                    # Por ahora, asignamos al nuevo documento
                    venta['idComprador'] = nuevo_documento
            
            mapeo_documentos[(num_doc, temp_id)] = nuevo_documento
            persona['numeroDocumento'] = nuevo_documento
            personas_dict[nuevo_documento] = persona
            compradores_en_ventas.add(nuevo_documento)
            nuevo_documento += 1
    
    # Paso 5: Eliminar identificadores temporales
    personas_finales = []
    for persona in personas_dict.values():
        if '_temp_id_persona' in persona:
            del persona['_temp_id_persona']
        if '_temp_doc_original' in persona:
            del persona['_temp_doc_original']
        personas_finales.append(persona)
    
    ventas_finales = []
    for venta in ventas_limpias:
        if '_temp_id_venta' in venta:
            del venta['_temp_id_venta']
        if '_temp_doc_original' in venta:
            del venta['_temp_doc_original']
        ventas_finales.append(venta)
    
    return personas_finales, ventas_finales


personas = requests.get("http://localhost:8080/mongo/personas").json()
ventas = requests.get("http://localhost:8080/mongo/ventas").json()

personas_limpias, ventas_limpias = filtrar_limpiar_personas(personas, ventas)

# Insertar en SQLite
for persona in personas_limpias:
    try:
        response = requests.post("http://localhost:8080/sqlite/personas", json=persona)
        if response.status_code != 200:
            print(f"Error al insertar persona {persona.get('idPersona')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar persona: {e}")

# Insertar en SQLite
for venta in ventas_limpias:
    try:
        response = requests.post("http://localhost:8080/sqlite/ventas", json=venta)
        if response.status_code != 200:
            print(f"Error al insertar venta {venta.get('idVenta')}: {response.json()}")
    except Exception as e:
        print(f"Excepción al insertar venta: {e}")

In [ ]:
# Obtener datos de MongoDB para ventas, escriba un ETL donde haga la validación de idComprador, idArticulo y cantidadProductos asegurando que solo contengan valores 
# numéricos, tener en cuenta elimiar el id que proporciona DBMongo a cada uno de las colecciones para cargarla a la Bases de Datos Relacional en SQLlite que importo.


# Obtener datos de MongoDB para ventas
response = requests.get("http://localhost:8080/mongo/ventas")
ventas = response.json()
ventas_limpias = filtrar_y_limpiar(ventas, 'idVenta', 'idComprador', 'idArticulo', 'cantidadProductos')
print(json.dumps(ventas_limpias, ensure_ascii=False, indent=2))

<font color="yellow" size = "4" face = "small fonts">6. Muestre la data almacenada en la base de datos Relacional que esta usando, tabla por tabla.</font></center><br>

In [ ]:
# Mostrar data almacenada en la tabla personas
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM personas;"

df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()


In [ ]:
# Mostrar data almacenada en la tabla articulos
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM articulos"
df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()

In [ ]:
# Mostrar data almacenada en la tabla ventas
# Conectar a la base de datos SQLite
conexion = sqlite3.connect('../data/data/almacen.sqlite')

# Crear una consulta SQL y cargar los resultados en un DataFrame
consulta = "SELECT * FROM ventas;"
df = pd.read_sql_query(consulta, conexion)

# Mostrar los datos
print(df)

# Cerrar la conexión
conexion.close()

<font color="yellow" size = "4" face = "small fonts">7. Grafique los 5 articulos mas vendidos.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

  # Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """SELECT
    a.idArticulo,
    a.nombreArticulo,
    SUM(v.cantidadProductos) AS unidades_vendidas,
    SUM(v.precioTotal)      AS ingresos_totales
    FROM ventas v
    JOIN articulos a ON a.idArticulo = v.idArticulo
    GROUP BY a.idArticulo, a.nombreArticulo
    ORDER BY unidades_vendidas DESC, ingresos_totales DESC, a.nombreArticulo ASC
    LIMIT 5;"""


df = pd.read_sql_query(consulta, conexion)

# Cerrar la conexión
conexion.close()

# Graficar los datos
plt.figure(figsize=(12, 6))
plt.bar(df['nombreArticulo'], df['unidades_vendidas'], color='skyblue')
plt.title('Top 5 Artículos Más Vendidos')
plt.xlabel('Artículo')
plt.ylabel('Unidades Vendidas')
plt.xticks(rotation=45)
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))

plt.tight_layout()
plt.show()



<font color="yellow" size = "4" face = "small fonts">8. Grafique los 5 compradores que han realizado más compras.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
  p.numeroDocumento,
  p.nombres,
  p.primerApellido,
  p.segundoApellido,
  COUNT(*)                           AS compras_realizadas,
  SUM(v.cantidadProductos)           AS unidades_compradas,
  SUM(v.precioTotal)                 AS total_gastado
FROM ventas v
JOIN personas p ON p.numeroDocumento = v.idComprador
GROUP BY p.numeroDocumento, p.nombres, p.primerApellido, p.segundoApellido
ORDER BY unidades_compradas DESC, total_gastado DESC, p.primerApellido ASC, p.nombres ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)


# Asegura tipos y arma el nombre completo de forma segura
# Crear nombre completo del cliente
df['cliente'] = (df['nombres'] + ' ' + df['primerApellido'] + ' ' + df['segundoApellido']).str.strip()

# Graficar
plt.figure(figsize=(12, 6))
plt.bar(df['cliente'], df['total_gastado'], color='lightcoral')
plt.title('Top 5 Compradores que Más Han Comprado')
plt.xlabel('Cliente')
plt.ylabel('Total Gastado ($)')
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.show()

# Cerrar la conexión
conexion.close()


<font color="yellow" size = "4" face = "small fonts">9. Grafique la distribución de precios de los 5 primeros artículos .</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
  idArticulo,
  nombreArticulo,
  precioArticulo
FROM articulos
ORDER BY idArticulo ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)

# Graficar
plt.figure(figsize=(10, 6))
plt.bar(df['nombreArticulo'], df['precioArticulo'], color='lightgreen')
plt.title('Distribución de Precios de los 5 Primeros Artículos')
plt.xlabel('Artículo')
plt.ylabel('Precio ($)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Cerrar la conexión
conexion.close()


<font color="yellow" size = "4" face = "small fonts">10. Grafique los 5 artículos que menos se han vendido.</font></center><br>

In [ ]:
# Conectar a la base de datos
conexion = sqlite3.connect('../data/data/almacen.sqlite') 

# Ejecutar la consulta y cargar los datos en un DataFrame
consulta = """
SELECT
    a.idArticulo,
    a.nombreArticulo,
    COALESCE(SUM(v.cantidadProductos), 0) AS unidades_vendidas,
    COALESCE(SUM(v.precioTotal), 0) AS ingresos_totales
FROM articulos a
LEFT JOIN ventas v ON a.idArticulo = v.idArticulo
GROUP BY a.idArticulo, a.nombreArticulo
ORDER BY unidades_vendidas ASC, ingresos_totales ASC, a.nombreArticulo ASC
LIMIT 5;
"""
df = pd.read_sql_query(consulta, conexion)

# Cerrar la conexión
conexion.close()

# Graficar los datos
plt.figure(figsize=(12, 6))
plt.bar(df['nombreArticulo'], df['unidades_vendidas'], color='salmon')
plt.title('Top 5 Artículos Menos Vendidos')
plt.xlabel('Artículo')
plt.ylabel('Unidades Vendidas')
plt.xticks(rotation=45, ha='right')
plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.show()